In [1]:
#In this notebook we are going to train a model using seq2seq, that can be able to translate english to french sentences

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import re
import unicodedata
import os
import io
import time

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8488969163834871787
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12119147705101038056
physical_device_desc: "device: XLA_CPU device"
]


In [4]:
#Let's us download our dataset  fra-eng.zip now 

!git clone https://github.com/Tikquuss/meta_XLM

Cloning into 'meta_XLM'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 961 (delta 152), reused 135 (delta 71), pack-reused 723
Receiving objects: 100% (961/961), 33.25 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (546/546), done.


In [6]:
%cd meta_XLM
%env output_path=/content/data/para
! mkdir $output_path
! chmod +x ./scripts/anki.sh
!./scripts/anki.sh de,en deu-eng $output_path scripts/anki.py
!./scripts/anki.sh en,fr fra-eng $output_path scripts/anki.py

/content/meta_XLM
env: output_path=/content/data/para
mkdir: cannot create directory ‘/content/data/para’: No such file or directory
mkdir: cannot create directory ‘/content/data/para’: No such file or directory
--2020-10-15 12:26:20--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.108.196, 104.24.109.196, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8324970 (7.9M) [application/zip]
Saving to: ‘/content/data/para/deu-eng.zip’

deu-eng.zip         100%[===================>]   7.94M  19.4MB/s    in 0.4s    

2020-10-15 12:26:20 (19.4 MB/s) - ‘/content/data/para/deu-eng.zip’ saved [8324970/8324970]

Archive:  /content/data/para/deu-eng.zip
  inflating: /content/data/para/deu.txt  
  inflating: /content/data/para/_about.txt  
download and extract deu-eng data in /content/data/para
--2020-10-15 12:26:23--  http://www.ma

In [11]:
#Load data
!head -n 3 /content/data/para/en-fr.en.txt

Go.
Hi.
Hi.


In [15]:
!head /content/data/para/deu.txt

Go.	Geh.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)
Hi.	Hallo!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)
Hi.	Grüß Gott!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)
Run!	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)
Run.	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #941078 (Fingerhut)
Wow!	Potzdonner!	CC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122382 (Pfirsichbaeumchen)
Wow!	Donnerwetter!	CC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122391 (Pfirsichbaeumchen)
Fire!	Feuer!	CC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #1958697 (Tamy)
Help!	Hilfe!	CC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #575889 (MUIRIEL)
Help!	Zu Hülf!	CC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #2122375 (Pfirsichbaeumchen)
